In [27]:
import re
relation_map = {
    'cr2':'人物的出生日期',
    'cr4':'人物的出生地',
    'cr16':'人物的毕业院校',
    'cr20':'人物的配偶',
    'cr21':'人物的子女',
    'cr28':'组织机构的高管',
    'cr29':'组织机构的员工数',
    'cr34':'组织机构的创始人',
    'cr35':'组织机构的成立时间',
    'cr37':'组织机构的总部地点'
}
train = open('./train_shuffle', 'r',  encoding='utf-8')
lines  = train.readlines()

def process_data(lines):
    relations = []
    error = 0
    for line in lines:
        idx, text, relation = line.strip().split('\t')
        pattern1 = r'<e1>(.*?)</e1>'
        pattern2 = r'<e2>(.*?)</e2>'

        match = re.search(pattern1, text)
        e1_text = match.group(1)
        e1_pos = match.start()
        e1_end = match.end()

        match2 = re.search(pattern2, text)
        e2_text = match2.group(1)
        e2_pos = match2.start()
        e2_end =  match2.end()

        ## 判断e1与e2的先后位置
        if e1_pos < e2_pos:
            text = text.replace('<e1>', "")
            text = text.replace('</e1>', "")
            e1_end = e1_end - len('<e1>')-len("</e1>")
            e2_pos = e2_pos - len('<e1>')-len("</e1>")
            e2_end = e2_end - len('<e1>')-len("</e1>")
            text =text.replace('<e2>', "")
            text =text.replace('</e2>', "")
            e2_end = e2_end - len('<e2>')-len("</e2>")
        else:
            text = text.replace('<e2>', "")
            text = text.replace('</e2>', "")
            e2_end = e2_end - len('<e2>')-len("</e2>")
            e1_pos = e1_pos - len('<e2>')-len("</e2>")
            e1_end = e1_end - len('<e2>')-len("</e2>")
            text = text.replace('<e1>', "")
            text = text.replace('</e1>', "")
            e1_end = e1_end - len('<e1>')-len("</e1>")
        if text[e1_pos:e1_end] != e1_text or text[e2_pos:e2_end] != e2_text:
            error+=1
        relations.append({
            'text':text,
            'h':{
                'name':e1_text,
                'pos':[e1_pos, e1_end]
            },
            't':{
                'name':e2_text,
                'pos':[e2_pos, e2_end]
            },
            'relation':relation_map[relation]
        })
    return relations,error

In [28]:
import json
test_data = open('./test_1_q_labels.txt','r',encoding='utf-8')
test_data = test_data.readlines()
relations,err = process_data(test_data)

with open('./valid.txt', 'w', encoding='utf8') as f:
    for item in relations:
        f.write(json.dumps(item, ensure_ascii=False)+'\n')